# CORA (Basic Model)
### 2-layer GCN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


The following is implemented using Danish's GCN model which he sent us earlier in the quarter.

In [2]:
# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_feats, h_feats)
        self.conv2 = GCNConv(h_feats, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = self.conv1(x, edge_index)
        h = F.relu(h)
        h = self.conv2(h, edge_index)
        return h

# Load the datasets
cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = cora_dataset[0]
data = data.to(device)

In [3]:
in_feats = data.x.shape[1]
h_feats = 64
num_classes = cora_dataset.num_classes
model = GCN(in_feats, h_feats, num_classes)
model = model.to(device)

def train(model, data, train_mask, labels):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    logits = model(data.cuda())
    
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

### Uncomment the following cell if you want to save embeddings (Wyatt will likely do this, not sure - not my topic)

In [37]:
def save_embeddings_and_edges(m, d, dataset_name):
    """
    m: model
    d: data
    dataset_name: used to name the file
    """
    # change to evaluation mode
    m.eval()

    # turn off gradient
    with torch.no_grad():
        # get embeddings
        embeddings = m(d.cuda())

    # save embeddings
    np.save(f'../embeddings/{dataset_name}_embeddings.npy', embeddings.cpu().detach().numpy())

    # count number correct
    counterCorrect = int(0)

    # total number in test mask
    nonZero = d.test_mask.sum()

    # iterate over embeddings
    for i in range(0, len(embeddings)):
        # if embedding is in the test mask, look for the argmax
        # and compare it to y
        if d.test_mask[i] and torch.argmax(embeddings[i]) == d.y[i]:
            counterCorrect += 1
    print(f"embeddings are correct: {counterCorrect/nonZero}")

    # m.eval()
    # with torch.no_grad():
    #     #embeddings = model(g, features)
    #     #embeddings = model(data, in_feats)
    #     embeddings = m(d.cuda())
    # #print(embeddings.shape)
    # np.save(f'../embeddings/{dataset_name}_embeddings.npy', embeddings.cpu().detach().numpy())

    # counterCorrect = int(0)
    # nonZero = d.test_mask.sum()
    # # print(d.cuda().y)
    # # print(len(d.cuda().y))
    # # print(type(d.cuda().y[0]))
    # # acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    # for i in range(0, len(embeddings)):
    #     #print(f"{torch.argmax(embeddings[i])} == {data.cuda().y[i]}")
    #     print(data.test_mask[i])
    #     if data.test_mask[i] and torch.argmax(embeddings[i]) == y[i]:
    #         counterCorrect += 1
    # print(f"embeddings are correct: {counterCorrect/nonZero}")

    """
    # trying to figure out what embeddings is
    print(f"dtype: {embeddings.dtype}")
    print(f"device: {embeddings.device}")
    print(f"layout: {embeddings.layout}")
    print(f"stride: {embeddings.stride()}")

    
    # Printing Embeddings Tensor
    torch.set_printoptions(profile="full") # so it doesn' truncate the Tensor
    print(embeddings) # prints the whole tensor
    torch.set_printoptions(profile="default") # back to default
    """

    """
    # Printing the data matrix of type torch_geometric.data.data.Data
    print(f"data.cuda()'s type: {type(data.cuda())}") 
    print(f"data's type: {type(data)}") 
    for i in data.x:
        print()
        for j in i:
            print(f"{j}", end = ", ")
    """

    """
    # #edge_index = g.edges() # --- (original) g is not defined
    # #edge_index = embeddings.edges() # --- 'Tensor' object has no attribute 'edges'
    # #edge_index = data.edges() # --- 'GlobalStorage' object has no attribute 'edges'
    # # uncomment below lines when you figure out Tensors
    # edge_index = embeddings.cpu()
    # np.save(f'../embeddings/{dataset_name}_edge_index.npy', np.vstack((edge_index[0].numpy(), edge_index[1].numpy())))
    """

In [5]:
train(model, data, data.train_mask, data.y)


1.9524599313735962

In [6]:
def test():
    # data = cora_dataset[0].to(device)
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)

    # incorrect_indices = (pred[data.test_mask] != data.y[data.test_mask]).nonzero()

    # print("Incorrect Predictions Indices:", incorrect_indices.flatten().tolist())

    acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    return acc

for epoch in range(0, 1000):
    loss = train(model, data, data.train_mask, data.y)
    acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Accuracy: {acc:.4f}')

Epoch: 000, Loss: 1.6899, Accuracy: 0.6120
Epoch: 001, Loss: 1.3562, Accuracy: 0.7600
Epoch: 002, Loss: 1.0098, Accuracy: 0.7350
Epoch: 003, Loss: 0.7307, Accuracy: 0.7760
Epoch: 004, Loss: 0.5312, Accuracy: 0.7740
Epoch: 005, Loss: 0.3672, Accuracy: 0.7820
Epoch: 006, Loss: 0.2720, Accuracy: 0.7600
Epoch: 007, Loss: 0.1937, Accuracy: 0.7680
Epoch: 008, Loss: 0.1610, Accuracy: 0.7680
Epoch: 009, Loss: 0.1189, Accuracy: 0.7660
Epoch: 010, Loss: 0.1030, Accuracy: 0.7790
Epoch: 011, Loss: 0.0785, Accuracy: 0.7540
Epoch: 012, Loss: 0.0706, Accuracy: 0.7860
Epoch: 013, Loss: 0.0560, Accuracy: 0.7560
Epoch: 014, Loss: 0.0518, Accuracy: 0.7860
Epoch: 015, Loss: 0.0422, Accuracy: 0.7530
Epoch: 016, Loss: 0.0399, Accuracy: 0.7890
Epoch: 017, Loss: 0.0329, Accuracy: 0.7580
Epoch: 018, Loss: 0.0319, Accuracy: 0.7880
Epoch: 019, Loss: 0.0263, Accuracy: 0.7620
Epoch: 020, Loss: 0.0259, Accuracy: 0.7880
Epoch: 021, Loss: 0.0216, Accuracy: 0.7620
Epoch: 022, Loss: 0.0212, Accuracy: 0.7930
Epoch: 023,

In [7]:
torch.save(model.state_dict(), 'cora_gt.pt')

The following shows the 10 nodes with the highest degree.

In [8]:
# import networkx as nx
# from dgl import to_networkx
# import matplotlib.pyplot as plt

# G = to_networkx(g)
# pos = nx.spring_layout(G, seed=42)
# cent = nx.degree_centrality(G)
# node_size = list(map(lambda x: x * 500, cent.values()))
# cent_array = np.array(list(cent.values()))
# threshold = sorted(cent_array, reverse=True)[50]
# print("threshold", threshold)
# cent_bin = np.where(cent_array >= threshold, 1, 0.1)
# plt.figure(figsize=(12, 12))
# nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
#                                cmap=plt.cm.plasma,
#                                node_color=cent_bin,
#                                nodelist=list(cent.keys()),
#                                alpha=cent_bin)
# edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3)
# plt.show()

This shows me the node with the highest degree.

In [9]:
# threshold = sorted(cent_array, reverse=True)[0]
# print("threshold", threshold)
# cent_bin = np.where(cent_array >= threshold, 1, 0.1)
# plt.figure(figsize=(12, 12))
# nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
#                                cmap=plt.cm.plasma,
#                                node_color=cent_bin,
#                                nodelist=list(cent.keys()),
#                                alpha=cent_bin)
# edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3)
# plt.show()

In [38]:
save_embeddings_and_edges(model, data, 'cora')

embeddings are correct: 0.7560000419616699
